In [1]:
%run drop_test_simulator.ipynb
import pandas as pd

In [2]:
heli_direction = 180 # deg on unit circle
amount = 2500 # number of simulations of each possible scenario
time_step = 0.05 # s
nominal_airspeed=15.43334 # m/s, this is 30 knots

# note that if dt=0.25 the horizontal distance may be off by about 8 m (compared to dt=0.001)
def mc_init(num_sims, dt, mode, airspeed, wind_dir):
    all_coords = []
    v_wind_rndm = []
    v_plane_rndm = []
    v_side_ref = []
    init_pos_offset = []

    deg_wind_mean, deg_wind_std_dev  = (wind_dir, 10) # 20-40-60 degree cone (with probability 68-95-99.7)
    
    for i in range(num_sims):
        v_wind_rndm.append([np.random.rayleigh(2.47344), # rayleigh distro for wind magnitude at 10 m
                            np.random.normal(deg_wind_mean, deg_wind_std_dev)])
        
        actual_airspeed = np.random.normal(airspeed, 1.715) # +- 10 knot tolerance (3 sigma)
        
        # convert airspeed to ground speed by adding components of wind velocity at release altitude
        v_plane_rndm.append(actual_airspeed -
                            wind_profile(609.6/2, v_wind_rndm[-1][0]) *
                            np.cos(np.radians(v_wind_rndm[-1][1])))
        v_side_ref.append(wind_profile(609.6/2, v_wind_rndm[-1][0]) *
                          np.sin(np.radians(v_wind_rndm[-1][1])))
        
        init_pos_offset.append([np.random.normal(0, 30 * 0.3048), # converting feet to meters
                                np.random.normal(0, 30 * 0.3048),
                                np.random.normal(0, 15 * 0.3048)]) # account for the imperfections of reality
    print('random wind and helicopter vectors obtained\n')
    
    for i, vw in enumerate(v_wind_rndm):
        all_coords.append(trajectory(v_plane=v_plane_rndm[i],
                                     v_side=v_side_ref[i],
                                     deg_plane=heli_direction,
                                     v_wind=vw[0],
                                     deg_wind=vw[1],
                                     x_0_offset=init_pos_offset[i],
                                     mode=mode,
                                     dt=dt)[0])
        if i % 250 == 0: print('iterations:', i)
    print('done simulations!\n')
    
    return (all_coords, v_wind_rndm, v_plane_rndm)

In [3]:
# this function assumes dataset has not been translated,
# and hence should only be run once initially to determine CEP
def polar_coord(df):
    # magnitude of landing coordinate
    df['norm'] = df.loc[:,['x','y']].apply(np.linalg.norm, axis=1, raw=True)
    # azimuth angle of landing (note that this is on the unit circle)
    df['theta'] = df.loc[:,['x','y']].apply(lambda pos: np.arctan2(pos[1], pos[0]), axis=1, raw=True)
      
def handle_data(all_coords, v_wind_rndm, v_plane_rndm, mode):
    # landing coordinates
    # sims are conducted in metric units so we convert m to ft here.
    results_df = pd.DataFrame.from_records([np.array([m_to_ft(Xi) for Xi in X])
                                            for X in all_coords],
                                           columns=['x', 'y'])
    polar_coord(results_df)
    # plane ground velocities
    results_df['v_plane'] = [ms_to_knots(vel) for vel in v_plane_rndm]
    # wind magnitude
    results_df['v_w'] = [ms_to_knots(vel[0]) for vel in v_wind_rndm]
    # wind direction as on unit circle
    results_df['deg_w'] = [vel[1] for vel in v_wind_rndm]
    results_df['mode'] = [mode for vel in v_plane_rndm]
    print(results_df.describe())
    return results_df

In [4]:
# mission success
nominal_against_coords, nominal_against_wind, nominal_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# main chute failure
nomain_against_coords, nomain_against_wind, nomain_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='A', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# drogue failure
nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='B', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

# early deployment
early_against_coords, early_against_wind, early_against_plane = mc_init(
    num_sims=amount, dt=time_step, mode='C', airspeed=nominal_airspeed, wind_dir=heli_direction-180)

11.568187322078472 0.9136942025899079 2.7965456931286288
10.819034192659036 -0.09391584753028291 3.25443182156377
11.752270828311184 0.0034749011441464147 1.0293333218458192
6.410324674504136 0.3879683588469148 4.814438692415368
12.072533030256697 0.14705169632822174 1.589397229762163
6.5480085572655335 -1.0747216828036819 4.700553863193906
12.777266230430294 0.4208137866489596 3.6235039584140054
9.877762715711416 0.22005765211470693 2.4764910193850094
10.141325941189466 0.9330414812553532 2.6672867805736744
3.7171006072059676 -4.568311689433838 7.484828578080216
12.541490885194401 -0.4316933542508689 2.073101949362326
12.659499387965553 -0.6532385865315598 2.8835873546042086
15.531981757786788 -0.31463035384378746 1.6478668302055055
8.695724015890264 -1.4046843959549915 4.692937612133969
10.45610017289566 0.30312543933033287 2.518443510182232
12.647007224180621 0.6854465937828887 2.4164462198021357
11.927913523808341 -0.7878239640784755 2.004069281072448
13.661829673462439 0.623951803

11.007391689635508 -0.4121055018758853 4.447655498914009
12.82170728592764 0.2722686771700074 0.7675489225776384
12.469264141937042 -1.0895296947307707 2.8936531821572067
12.405944523478107 -0.2257942427590972 1.0928280595939874
6.202522602333532 -0.15578272816264876 4.609031857395263
8.970373105838743 1.1734786616263655 2.8530065112814933
10.29621158701691 0.48521697053347657 3.687019540062619
7.030286898769958 -0.5233997105757804 5.551590174487789
12.914788009511648 0.30639951256612186 1.2860382460145567
12.624220422365738 -0.4487541207902848 3.4283229865378755
11.599449712599835 -1.1649077523387286 3.5780111511910606
6.056283950198613 0.2243079546180254 3.8424775221388305
6.872674226060325 0.5307389973341589 3.9909397126620343
10.294263868757033 -0.021329710696220843 1.9722301082481193
10.503174968639346 -1.1545629814255332 3.7001651646153575
13.619809504277205 -0.548304738740301 1.1621759070118083
7.77550126641237 1.0800483829940462 3.759000916557964
14.758850570491608 -0.373333230

11.949609656411535 0.0480758025791683 1.6761869318218576
8.565595613688583 0.17177112901329908 1.7510888761686956
7.838296806987215 0.5153022639087868 4.204628801203667
6.8413609941115565 0.33853518636620755 3.699848492378562
15.58150337846559 0.524166316966966 0.9753437036449364
14.74985134740846 -0.3387187553209646 1.283284060520705
8.129837128325264 -1.2834441293302727 3.523177323587024
9.669760474210669 -0.6093683521972978 3.600553121951336
11.701331739456403 0.9742610542777526 2.8847272726286537
11.992738895666596 -0.9615804103027843 3.4907468328942572
10.524928076540448 0.3237319570153222 3.117526554219713
9.445403850215797 -0.1410317046412723 3.745450283200514
9.93557478277392 -0.8255503181624094 4.4315767049598564
12.451107369519455 0.17796210225419518 1.949385547921276
5.2134896689865435 2.13161441654002 8.120457634469766
15.251977573774516 -0.4448589262247939 0.7813341620528409
10.382420369587127 -0.46319417308420596 4.141861881016957
16.306076930681105 0.07581430619843478 0.

15.127287394429198 -0.09306941160125559 0.9942335430447452
13.096941451256455 0.11051536026070027 2.4923801233287834
13.294062466693935 0.4150555994258833 0.836199741976137
10.122831809488396 0.11407060214118954 1.582341744213504
7.84406248231264 1.6234106266066284 3.8080862841700447
14.435541874285676 0.3468289222530862 3.80051810762344
12.012156366888263 0.8677387999296382 3.116789104815252
9.139019273025724 0.36030678720624054 2.567040485177509
11.442054445666873 1.8187310171000917 2.48988799523038
10.594482992654628 -0.4408283603358084 3.6743973136663906
14.611275303041033 0.05754930558444403 0.7415378130412447
4.243035354784615 1.2574343666623764 5.322424804862594
15.283376660406404 0.2271487574462593 0.9711710025378002
14.069126301871561 0.06644136155503305 2.450324982172138
6.807844273806501 3.0786439935081304 5.110595993586311
9.443176777136845 0.7821198148478965 5.213907973637659
7.700111024985817 -0.15053676678406247 4.172905526246032
10.475428542341888 -0.8013869515463911 3.

iterations: 0


KeyboardInterrupt: 

In [ ]:
# mission success
nominal_across_coords, nominal_across_wind, nominal_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# main chute failure
nomain_across_coords, nomain_across_wind, nomain_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='A', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# drogue failure
nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='B', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# early deployment
early_across_coords, early_across_wind, early_across_plane = mc_init(
    num_sims=amount, dt=time_step, mode='C', airspeed=nominal_airspeed, wind_dir=heli_direction-90)

# Data Handling for Against Wind Simulations

In [ ]:
nominal_against_data = handle_data(nominal_against_coords, nominal_against_wind, nominal_against_plane, 'nominal')
nomain_against_data = handle_data(nomain_against_coords, nomain_against_wind, nomain_against_plane, 'nomain')
nodrogue_against_data = handle_data(nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane, 'nodrogue')
early_against_data = handle_data(early_against_coords, early_against_wind, early_against_plane, 'early')

# Data Handling for Across Wind Simulations

In [ ]:
nominal_across_data = handle_data(nominal_across_coords, nominal_across_wind, nominal_across_plane, 'nominal')
nomain_across_data = handle_data(nomain_across_coords, nomain_across_wind, nomain_across_plane, 'nomain')
nodrogue_across_data = handle_data(nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane, 'nodrogue')
early_across_data = handle_data(early_across_coords, early_across_wind, early_across_plane, 'early')

## Saving data and Aggregate Results

In [ ]:
nominal_against_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_against_data.csv')
nomain_against_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_against_data.csv')
nodrogue_against_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_against_data.csv')
early_against_data.to_csv(path_or_buf='./drop_test_sample_data/early_against_data.csv')

In [ ]:
nominal_across_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_across_data.csv')
nomain_across_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_across_data.csv')
nodrogue_across_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_across_data.csv')
early_across_data.to_csv(path_or_buf='./drop_test_sample_data/early_across_data.csv')